ЗАДАНИЕ 3

В базе данных вашей компании присутствуют следующие таблицы

city

city_id – id города, где проходит акция
client_city_id – id города, где находится покупатель
city – город
client_city – город, где находится покупатель
client
client_id – id покупателя
client_city_id – id города, где находится покупатель
birth_date – дата рождения покупателя
registration – дата регистрации покупателя
promotion

promotion_id – id акции
category_id – id категории
promotion_name – акция
category_name – категория
partner_id – id партнёра
partner_name – партнёр

purchase

purchase_id – id покупки
partner_id – id партнёра
client_id – id покупателя
city_id – id города
promotion_id – id акции
category_id – id категории
purchase_date – дата покупки
price – цена за единицу товара
quantity – число проданных единиц
status – статус покупки

Вам нужно написать запрос, чтобы получить такую таблицу

purchase_date – дата покупки
purchase_id – id покупки
client_id – id покупателя
client_age – возраст покупателя
client_registration_age – как долго человек пользуется вашими магазинами
client_category – содержит new или old, если это первая или последующая покупка соответственно
promotion_name – акция
category_name – категория
partner_name – партнёр
client_city – город, где находится покупатель
city – город
revenue – сумма выручки
quantity – число проданных единиц

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

In [20]:
import pandas as pd
import pandahouse as ph

In [21]:
connection_default = dict(host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

In [37]:
query = '''
SELECT 
    purch.purchase_date as purchase_date, 
    purch.purchase_id as purchase_id, 
    purch.client_id as client_id,
    dateDiff('year', client.birth_date, today()) as client_age, 
    datediff('year',  client.registration, today()) as client_registration_age, 
    promo.promotion_name as promotion_name, 
    promo.category_name as category_name, 
    promo.partner_name as partner_name, 
    city.client_city as client_city, 
    city.city as city, 
    toFloat32(purch.price) *purch.quantity as revenue, 
    toFloat32(purch.quantity) as quantity
FROM
(
    SELECT * 
    FROM 
        project_variant_2.purchase
    WHERE
        status = 1 and 
        purchase_date >= '2020-05-01' and 
        purchase_date <= '2020-08-01') as purch 

JOIN 
    project_variant_2.city as city 
ON 
    purch.city_id = city.city_id 

JOIN
    project_variant_2.client as client
ON 
    purch.client_id = client.client_id 

JOIN 
    project_variant_2.promotion as promo 
ON 
    purch.category_id = promo.category_id
'''
df = ph.read_clickhouse(query, connection=connection_default)

In [38]:
df.head()

,purchase_date,purchase_id,client_id,client_age,client_registration_age,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-06-05,0,853,29,2,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Киев,150.0,5.0
1,2020-06-05,0,853,29,2,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Москва,Киев,150.0,5.0
2,2020-06-05,0,853,29,2,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Ярославль,Киев,150.0,5.0
3,2020-06-05,0,853,29,2,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Санкт-Петербург,Киев,150.0,5.0
4,2020-06-05,0,853,29,2,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Новгород,Киев,150.0,5.0


In [39]:
df.shape

(23479, 12)